## EDA. Red Wine Quality

### *Выполнила Вощинина Мария*

Ссылка на каггл: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

In [ ]:
%matplotlib inline

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

### Загрузка данных

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

In [ ]:
df.shape

In [ ]:
df.head(7)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
           'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
           'pH', 'sulphates', 'alcohol']
target = ['quality']

### Инсайды с Pandas-Profiling

*По совету с лекции впервые использовала библиотку. Интресено, но не все понравилось (например, корреляция между переменными шлчит и не информативна как-то)*

* есть дубликаты(15%)
* нет попущенных значений, возможно их уже заполнили. Далее попробую проверить эту гипотезу
* citric_acid содержит 8% нулей, возможно ими заполнили пропущенные значения!
* есть подозрительные значения у total sulfur dioxide и free sulfur dioxide, sulphates
* фактически оценки ставили от 3 до 8,на оценки 5 и 6 приходится 83% наблюдений
* total sulfur dioxide и free sulfur dioxide сильно коррелируют, также fixed acidity, volatile acidity (что логично)

In [ ]:
pandas_profiling.ProfileReport(df)

### Boxplot

Выводы:
* по некоторым признакам можно увидеть, что оценка качества  линейно зависит от значений признака. Например, volatile acidity, citric acid,
* у некоторых признаков, например residual sugar, много выбросов. Можно поробовать обработать этот признак
* еще можно заметить, что выбросы в основном по оценкам качества 5 и 6

In [ ]:
for i in features:
    plt.figure(i, figsize=(7,7))
    sns.set_palette("Blues")
    ax = sns.boxplot(x="quality", y=i, data=df) 
    plt.title('Boxplot для %s' %i) 

### Гистограммы с большим количеством бинов

Выводы:
* была гипотеза, что пропуски заполняют каким то значением.Она не подтвердилась
* распределения некоторых величин похожи на нормальное. Например, pH
* Некоторые переменные стоит прологарифмировать

In [ ]:
for i in features:
    plt.hist(df[i], 70, alpha=0.75, color ='k')
    plt.xlabel('%s' %i)
    plt.title('Гистограмма %s' %i)
    plt.grid(True)
    plt.show()

### Построим логарифмы для некоторых переменных

Выводы:
   * после логарифмирования распределение признака chlorides стало похоже на нормальное.

In [ ]:
log = ['residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide']
for i in log:
    plt.hist(np.log(df[i]), 70, alpha=0.75, color ='k')
    plt.xlabel('%s' %i)
    plt.title('Гистограмма log(%s)' %i)
    plt.grid(True)
    plt.show()

### Гистограммы с разрезом по качеству

Выводы:
* По сути, этим графиком проверяются те же гипотезы, что и boxplot. В данном случае  много классов, поэтому график не очень удобно читать и тяжело увидеть какие-то закономерности. 

In [ ]:
for i in features:
    for j in list(set(df['quality'])):       
        subset = df[df['quality'] == j]
        plt.figure(i, figsize=(15,3))
        sns.set_palette("Blues")
        ax = sns.distplot(subset[i],  hist = False, kde = True,
                  kde_kws = {'linewidth': 3}, 
                  label = j)        
        plt.legend(prop={'size': 16}, title = 'quality')
        plt.title('Гистограмма для %s' %i)
        plt.xlabel('%s' %i)

### Зависимость между качеством и признаками

Построим на 50% выборки

Выводы:
* Опять же, boxplot удобнее. Выводы такие же

In [ ]:
for i in features:
    #plt.figure(i, figsize=(7,7))
    sns.set_palette("Blues", 1)
    ax = sns.jointplot(y=i, x="quality", data=df.sample(frac=0.5))
    plt.title('Диаграмма рассеяния для %s' %i, loc='center')
    #plt.subplots_adjust(hspace = 3)
    plt.xlabel('%s' %i)

### Диаграмма параллельных координат (?)

Выводы:

* Можно заметить, что есть переменные, для которых качество 3 и качество 8 находятся в противоположных концах. Такие переменные:
    *  volatile acidity
    *  citric acid
    * chlorides
    * pH
    * sulphates
    * alcohol
    
    Полагаю, что они как раз хорошо себя покажут в классификации

In [ ]:
fig = px.parallel_coordinates(df.sample(frac=0.5),color="quality", color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

Вышло не очень наглядно. Попробуем построить средние значения

In [ ]:
df_means = pd.DataFrame(df.groupby('quality', as_index = False).agg('mean'))
df1 = df_means.pop('quality')
df_means['quality'] = df1
df_means.head()

In [ ]:
fig = px.parallel_coordinates(df_means,color="quality", color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

Еще хотела добавить на график sd, но не получилось

### Лепестковая диаграмма

*https://medium.com/nuances-of-programming/4-%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%8B%D0%B5-%D0%B2%D0%B8%D0%B7%D1%83%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D0%B8-%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85-%D0%B2-python-%D1%81-%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E-%D0%BA%D0%BE%D0%B4%D0%B0-ca58253fa1a3*

Строим на основе таблицы из пункта выше, плюс нормируем все признаки.

Выводы:
  * в целом, выводы такие же как и в пункте выше
  * на таком графике хорошо видно, увеличение каких показателей приводит к ухудшению/улучшению качества вина

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1)) 
df_means_scaled = pd.DataFrame(scaler.fit_transform(df_means[features]), columns = features)
df_means_scaled['quality'] = df_means['quality']

In [ ]:
labels = features
stats=df_means_scaled.loc[5,labels].values
stats0=df_means_scaled.loc[0,labels].values

angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
stats = np.concatenate((stats,[stats[0]]))
stats0 = np.concatenate((stats0,[stats0[0]]))
angles = np.concatenate((angles,[angles[0]]))

# Plot stuff
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, polar=True)
ax.plot(angles, stats, 'o-', linewidth=2, color = 'r')
ax.fill(angles, stats, alpha=0.2, color = 'r')
ax.plot(angles, stats0, 'o-', linewidth=2, color = 'g')
ax.fill(angles, stats0, alpha=0.2, color = 'g')
ax.set_thetagrids(angles * 180/np.pi, labels)
ax.set_title("Сравнение вин разного качества")
ax.legend(('quality = 8', 'quality = 3'), loc =1)
ax.grid(True)
plt.show()